AUTENTICAÇÃO DA API TWITTER

In [2]:
import os
import requests
from dotenv import load_dotenv

# Verificar o caminho atual do diretório
current_directory = os.getcwd()
print(f"Diretório atual: {current_directory}")

# Verificar se o arquivo .env está presente no diretório atual
env_path = os.path.join(current_directory, '.env')
if not os.path.exists(env_path):
    raise FileNotFoundError(f"Arquivo .env não encontrado no diretório: {current_directory}")

# Carregar variáveis de ambiente
load_dotenv(env_path, override=True)

# Acessar as variáveis de ambiente
BEARER_TOKEN = os.getenv('BEARER_TOKEN')

# Verificar se a variável de ambiente foi carregada corretamente
if not BEARER_TOKEN:
    raise ValueError("A variável de ambiente BEARER_TOKEN não foi carregada corretamente.")

# Função para buscar tweets usando a API v2
def buscar_tweets_saude_v2(query, quantidade=10):
    url = "https://api.twitter.com/2/tweets/search/recent"
    headers = {
        "Authorization": f"Bearer {BEARER_TOKEN}"
    }
    params = {
        "query": query,
        "max_results": quantidade,
        "tweet.fields": "author_id,created_at,text,geo",
        "expansions": "geo.place_id",
        "place.fields": "full_name,id,contained_within,country,country_code,geo,name,place_type"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"Erro ao buscar tweets: {response.status_code} - {response.text}")
    return response.json()

# Palavras-chave relacionadas à saúde
palavras_chave_saude = ["saúde", "hospital", "remédio", "vacina", "medicamento", "UPA", "SUS", "UBS", "Posto de Atendimento", "Medico", "Enfermeiro", "Farmácia", "Consultório", "Pronto Socorro", "Clinica", "Emergência", "Pandemia", "Epidemia", "Covid", "Coronavírus", "Vacinação", "Imunização", "Epidemiologia", "Dengue", "Crise Hospitalar", "Doença"]

# Construir a query combinando as palavras-chave e filtrando por localização
query = " OR ".join([f'"{palavra}"' for palavra in palavras_chave_saude]) + ' place:"Brasília"'

# Buscar tweets relacionados à saúde
tweets_saude = buscar_tweets_saude_v2(query, quantidade=10)

# Exibir os tweets coletados
for tweet in tweets_saude['data']:
    print(f"Usuário: {tweet['author_id']}, Texto: {tweet['text']}, Data: {tweet['created_at']}")

Diretório atual: c:\Users\Sony\Documents\crudBasico\ProjetodeTCC_AnalysSentimento\analise_sentimentos\coleta_dados
Autenticação bem-sucedida
Erro ao coletar tweets: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product


COLETANDO DADOS COM QUERYS - DOMÍNIO SAÚDE E SUBDOMÍNIOS --- Descobrindo principais tendências -- Definição do que será captado.

In [3]:
import os
import tweepy
from dotenv import load_dotenv
import time

# Carregar variáveis de ambiente
load_dotenv()

# Acessar as variáveis de ambiente
API_KEY = os.getenv('API_KEY')
API_SECRET_KEY = os.getenv('API_SECRET_KEY')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('ACCESS_TOKEN_SECRET')

# Código de autenticação
autenticaTweet = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
autenticaTweet.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(autenticaTweet)

# Palavras-chave relacionadas à saúde
palavras_chave_saude = ["saúde", "hospital", "remédio", "vacina", "medicamento", "UPA", "SUS", "UBS", "Posto de Atendimento", "Medico", "Enfermeiro", "Farmácia", "Consultório", "Pronto Socorro", "Clinica", "Emergência", "Pandemia", "Epidemia", "Covid", "Coronavírus", "Vacinação", "Imunização", "Epidemiologia", "Dengue", "Crise Hospitalar", "Doença"]

# Construir a query combinando as palavras-chave
query = " OR ".join([f'"{palavra}"' for palavra in palavras_chave_saude])

# Coordenadas de Brasília
latitude = -15.7942
longitude = -47.8822
raio = "50km"  # Defina o raio de busca ao redor de Brasília

# Função para coletar tweets com base nas restrições do plano livre
def coletar_tweets_saude(quantidade=10):
    resultados = []
    try:
        # Limite de 1 solicitação por minuto com 10 tweets por chamada para não ultrapassar o limite mensal
        tweets = tweepy.Cursor(api.search_tweets,
                               q=query,
                               geocode=f"{latitude},{longitude},{raio}",
                               tweet_mode='extended',
                               lang="pt",
                               result_type="recent"
                              ).items(quantidade)
        
        # Armazenar tweets coletados
        for tweet in tweets:
            resultados.append({
                "usuario": tweet.user.screen_name,
                "texto": tweet.full_text,
                "data": tweet.created_at
            })
            print(f"Coletado tweet de @{tweet.user.screen_name}: {tweet.full_text[:50]}...")
        
        print(f"\nTotal de tweets coletados nesta execução: {len(resultados)}")
        return resultados

    except tweepy.TweepyException as e:
        print(f"Erro ao coletar tweets: {e}")
        time.sleep(60)  # Esperar um minuto caso ocorra erro de rate limit
        return resultados

# Executar a função de coleta e limitar para evitar ultrapassar o limite mensal
tweets_saude = coletar_tweets_saude(quantidade=10)  # Coleta de 10 tweets por execução

Erro ao coletar tweets: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product


In [21]:
print(tweets_saude)

[]
